In [6]:
import os
from glob import glob
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

from IPython.display import display

In [7]:
import src.elevation as elevation
import src.clean as clean
import src.plot as plot

In [8]:
# Paths
shapefile_bisse = os.path.join("data", "bisses_geodata", "bisses.shp")
dem_folder = os.path.join("data", "DEM")  # Folder containing multiple DEM files
data = "data"

# ! read from json file
# path of file from field results
PATH = os.path.join(data, "bisses")
print(PATH)
# get all json of bisse
folders = [f.path for f in os.scandir(PATH) if f.is_dir()]
files = [y for x in os.walk(PATH) for y in glob(os.path.join(x[0], "*.json"))]
filenames = [Path(path).stem for path in files]

# ! check if file is missing for a folder
# all bisse where data has been collected in a json
bisses_dict = dict(zip(filenames, files))
print(bisses_dict)
# this is the variable the interface will change
bisse_select = "clavau"

# json holding the info for that specific bisse
# ie path, sample dir, date, ect...
with open(bisses_dict[bisse_select], "r") as file:
    bisse_json = json.load(file)

# json holding info for each entry that is common to all bisses
with open(os.path.join(data, "_template", "donnees.json"), "r") as file:
    generic_json = json.load(file)
clean.generic_json = generic_json

# display(bisse_json)
# bisse_json = bisses_json[bisse_select]

data\bisses
{'clavau': 'data\\bisses\\clavau\\clavau.json', 'plot': 'data\\bisses\\clavau\\touristique\\plot.json'}


In [9]:
# Load the line shapefile
bisses_gdf = gpd.read_file(shapefile_bisse).to_crs("EPSG:2056")  # to swiss projection

# get only the bisse we want
bisse_select_gdf = bisses_gdf[bisses_gdf["Nom_bisse"] == bisse_json["nom_inventaire"]]
# extract the linestring
bisse_line = bisse_select_gdf.loc[:, "geometry"].iloc[0]

bisse_profile = elevation.get_elevation_profile(bisse_line, dem_folder=dem_folder)

# cleaning data referenced in json

In [10]:
path_clavau = os.path.normpath("data/bisses/clavau")

# #! need to do that for all couches
clean.clean_list_json(
    bisse_json,
    "liste_touristique",
    bisse_line,
    "data/bisses/clavau/touristique",
)
clean.clean_list_json(
    bisse_json,
    "liste_environnement",
    bisse_line,
    "data/bisses/clavau/environnement",
)

clean.clean_list_json(
    bisse_json,
    "liste_agricole",
    bisse_line,
    "data/bisses/clavau/agricole",
)

clean.clean_list_json(
    bisse_json,
    "liste_hydrologique",
    bisse_line,
    "data/bisses/clavau/hydrologique",
)

d:\Programme\miniconda3\envs\bisses\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'PointM' is converted to 'Point'
  return ogr_read(
